In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.359186,"""a""",0.056239,0.155466,0.044328,0.625709,0.858837,-0.011383
0.841471,1,"""a""",0,0.92849,"""a""",0.776532,0.411995,0.185399,0.085683,0.057331,-0.037931
0.909297,2,"""a""",1,0.439717,"""a""",0.330252,0.351294,0.5227,0.08072,0.557959,-0.629124
0.14112,3,"""a""",0,0.996919,"""a""",0.377956,0.123035,0.686929,0.448204,0.415162,-0.93677
-0.756802,4,"""a""",0,0.025227,"""a""",0.303869,0.822485,0.412584,0.765767,0.272874,-0.326544


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.query_jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.625709,-0.858837
-0.841471,-0.085683,-0.057331
-0.909297,-0.08072,-0.557959
-0.14112,-0.448204,-0.415162
0.756802,-0.140058,0.585963


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[-0.490676, -0.361612]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975
str,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.218322,0.001686,129.51111,0.0,0.215017,0.221626
"""exp(x2)""",0.17469,0.000679,257.413749,0.0,0.17336,0.17602
"""sin(x3)""",-1.743507,0.00134,-1300.649075,0.0,-1.746135,-1.74088
"""__bias__""",-0.108079,0.001497,-72.185358,0.0,-0.111013,-0.105144


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[-0.490676, -0.361612]"


In [10]:
df.select(
    "dummy",
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.485351, -0.364542]"
"""a""","[-0.485351, -0.364542]"
"""a""","[-0.485351, -0.364542]"
"""a""","[-0.485351, -0.364542]"
"""a""","[-0.485351, -0.364542]"
…,…
"""b""","[-0.496067, -0.358636]"
"""b""","[-0.496067, -0.358636]"
"""b""","[-0.496067, -0.358636]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.query_lstsq(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.056239,0.155466,-0.011383,-0.083813,0.07243
0.776532,0.411995,-0.037931,-0.530008,0.492077
0.330252,0.351294,-0.629124,-0.289079,-0.340045
0.377956,0.123035,-0.93677,-0.229945,-0.706825
0.303869,0.822485,-0.326544,-0.446522,0.119977


In [12]:
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.496067, -0.358636]"
"""a""","[-0.485351, -0.364542]"


In [13]:
# Lasso
df.group_by("dummy").agg(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""b""","[-0.323816, -0.185978]"
"""a""","[-0.31474, -0.192814]"


In [14]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.query_lstsq(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.541371
"""a""",-0.515024


In [15]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.query_rolling_lstsq(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.011383,0.056239,0.155466,null,null
-0.037931,0.776532,0.411995,null,null
-0.629124,0.330252,0.351294,null,null
-0.93677,0.377956,0.123035,null,null
-0.326544,0.303869,0.822485,"[-0.576516, -0.193246]",-0.334127
…,…,…,…,…
-1.259886,0.034561,0.752261,"[0.282414, -1.23973]",-0.922841
-0.20819,0.040898,0.021563,"[0.614343, -1.462836]",-0.006418
-0.667431,0.250875,0.402446,"[0.648749, -1.760264]",-0.545656


In [16]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [17]:
# Only want singular values (principal values?)
df.select(
    pds.query_singular_values("a", "b", "x1")
)

a
list[f64]
"[29.227034, 28.841191, 28.673175]"


In [18]:
# Singular values + The principal components
df.select(
    pds.query_pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
29.213465,"[0.780529, -0.625119]"
28.729148,"[0.625119, 0.780529]"


In [19]:
# PC1
df.select(
    pds.query_principal_components("a", "b", k =1)
).unnest("principal_components").head()

pc1
f64
-0.123833
-0.044303
-0.361128
0.014969
0.351783


# ML Metrics

In [20]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.333998,1.00988,0.501614,0.49345,0.497498,0.505839,0.499459
"""a""",0.339075,1.012761,0.500822,0.474669,0.487395,0.501693,0.488649


# Str Extension

In [21]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [22]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""hello"""
"""world"""
"""church"""
"""to"""
"""going"""


In [23]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""church"""
"""go"""
""""""
"""world"""
"""hello"""


In [24]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [25]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [26]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [27]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [28]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [29]:
df.filter(
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [30]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [31]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.868297
0.552773
-1.152573


In [32]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,0.744406,null
null,0.936033,null
-0.868297,0.556436,0.723798
0.552773,1.445886,0.453344
-1.152573,-1.195777,-0.006817


In [33]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""Pi3""",null
null,"""ZCnaW""",null
-0.868297,"""2""","""2"""
0.552773,"""e""","""e"""
-1.152573,"""7mg3Z""","""7mg3Z"""


In [34]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.868297,"""IeA07"""
0.552773,"""3ZtJz"""
-1.152573,"""NoAKh"""


In [35]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.314265,null,-0.314012
null,-1.438597,null,-1.438247
-0.868297,-0.387446,0.37435,-0.387077
0.552773,-2.335712,2.323368,-2.336171
-1.152573,-1.928985,0.281368,-1.92879


In [36]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.750827,2.036969,5
null,0.345218,0.486922,2
-0.868297,0.537081,0.257583,3
0.552773,0.07438,-1.313862,3
-1.152573,0.702153,0.096594,5


In [37]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.query_ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.766988,0.443215,0.75449,0.685748


In [38]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.968969,0.482669,2,1
1,0.454712,0.500958,4,9
2,0.275668,0.37223,4,6
0,0.244416,0.467948,0,3
1,0.14292,0.54059,1,1


In [39]:
# In dataframe statistical tests!
df.select(
    pds.query_ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.query_chi2("category_1", "category_2").alias("chi2-test"),
    pds.query_f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.242506,0.808393}","{30.627761,0.721731}","{1.144874,0.333362}"


In [40]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.query_ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.query_chi2("category_1", "category_2").alias("chi2-test"),
        pds.query_f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {-0.508801,0.610925} ┆ {46.709664,0.108988} ┆ {1.720471,0.14282}  │
│ 1         ┆ {0.493627,0.621602}  ┆ {39.48074,0.317128}  ┆ {0.318692,0.865595} │
│ 2         ┆ {0.437398,0.661851}  ┆ {24.599585,0.924818} ┆ {0.553822,0.696258} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [41]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
557,0.1114
583,0.1166
573,0.1146
540,0.108
526,0.1052
572,0.1144
537,0.1074
553,0.1106
559,0.1118


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [42]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [43]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.973253,0.784598,0.032257,0.413564,5.882922,8
1,0.964942,0.934959,0.125431,0.566296,1.299944,10
2,0.767643,0.811002,0.110048,0.112519,3.814312,17
3,0.097368,0.929946,0.176087,0.493587,3.610233,11
4,0.574719,0.092724,0.530183,0.593004,1.332709,17


In [44]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.973253,0.784598,0.032257,0.413564,5.882922,56
1,0.964942,0.934959,0.125431,0.566296,1.299944,133
2,0.767643,0.811002,0.110048,0.112519,3.814312,5
3,0.097368,0.929946,0.176087,0.493587,3.610233,146
4,0.574719,0.092724,0.530183,0.593004,1.332709,367


In [45]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.973253,0.784598,0.032257,0.413564,5.882922,"[0, 1569, … 709]"
1,0.964942,0.934959,0.125431,0.566296,1.299944,"[1, 339, … 1014]"
2,0.767643,0.811002,0.110048,0.112519,3.814312,"[2, 1106, … 1819]"
3,0.097368,0.929946,0.176087,0.493587,3.610233,"[3, 53, … 213]"
4,0.574719,0.092724,0.530183,0.593004,1.332709,"[4, 1174, … 642]"


In [46]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1569, … 709]  ┆ 3                  │
│ 1   ┆ [1, 339, … 709]   ┆ 7                  │
│ 2   ┆ [2, 1106, … 902]  ┆ 8                  │
│ 3   ┆ [3, 53, … 1316]   ┆ 6                  │
│ 4   ┆ [4, 1174, … 1498] ┆ 9                  │
└─────┴───────────────────┴────────────────────┘


In [47]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.973253,0.784598,0.032257,0.413564,5.882922,"[0, 1569, … 709]","[0.0, 0.064215, … 0.096617]"
1,0.964942,0.934959,0.125431,0.566296,1.299944,"[1, 339, … 1014]","[0.0, 0.036057, … 0.063321]"
2,0.767643,0.811002,0.110048,0.112519,3.814312,"[2, 1106, … 1819]","[0.0, 0.029892, … 0.062706]"
3,0.097368,0.929946,0.176087,0.493587,3.610233,"[3, 53, … 213]","[0.0, 0.021567, … 0.070992]"
4,0.574719,0.092724,0.530183,0.593004,1.332709,"[4, 1174, … 642]","[0.0, 0.05006, … 0.08659]"


In [48]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
4,0.574719,0.092724,0.530183,0.593004,1.332709
5,0.284424,0.271062,0.553531,0.417966,4.595913
6,0.765066,0.559227,0.793738,0.970897,7.312772
7,0.215392,0.601253,0.42271,0.857596,2.016605
10,0.487338,0.234103,0.457489,0.482822,6.715263


In [49]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
13,0.554396,0.488217,0.470983,0.678725,7.485183
52,0.559481,0.56312,0.321518,0.944206,8.376489
74,0.550888,0.496313,0.674324,0.696142,5.191851
92,0.466705,0.528252,0.255016,0.897756,8.242887
93,0.555664,0.489169,0.348829,0.424842,0.734708


In [50]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
13,0.554396,0.488217,0.470983,0.678725,7.485183
92,0.466705,0.528252,0.255016,0.897756,8.242887
160,0.463381,0.427564,0.176633,0.604385,9.981861
298,0.440254,0.535504,0.717983,0.059325,9.33173
496,0.525596,0.449178,0.219502,0.568355,8.999516


In [51]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,[0],1
1,"[1, 1533, 339]",3
2,"[2, 788]",2
3,"[3, 53, … 1592]",4
4,"[4, 533]",2


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [52]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [53]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 1, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [54]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [55]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pds.query_similar_words(
        "a",
        vocab = pl.col("b"),
        k = 2, 
        threshold = 4,
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [56]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pds.query_similar_words(
        "a",
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3, 
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"


# Using PDS Expressions On Series / NumPy arrays

In [57]:
df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
)
df.head()

actual,predicted,0-2,0-9
f64,f64,i32,i32
0.0,0.072027,0,5
0.0,0.592545,1,7
1.0,0.232183,1,7
1.0,0.715533,0,1
1.0,0.289555,2,7


In [58]:
pds.eval_series(
    df["0-2"], df["0-9"], # use series as args
    expr = "query_jaccard_col" # name of the pds expression
)

jaccard_col
f64
0.3


In [59]:
pds.eval_series(
    df["actual"], df["predicted"], # use series as args
    expr = "query_binary_metrics" # name of the pds expression
).unnest("binary_metrics")

precision,recall,f,average_precision,roc_auc
f64,f64,f64,f64,f64
0.49979,0.50012,0.499955,0.50036,0.502358


In [60]:
pds.eval_series(
    # can also use NumPy
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    expr = "query_psi", # name of the pds expression
    n_bins = 5, 
    return_report = True
).unnest("psi")

cnt<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.16953,0.2,0.162,0.008007
0.359035,0.2,0.197,0.000045
0.572823,0.2,0.224,0.00272
0.799598,0.2,0.23,0.004193
inf,0.2,0.187,0.000874


In [61]:
pds.eval_series(
    np.random.random(size = 1000), 
    expr = "query_cid_ce", # name of the pds expression
)

cid_ce
f64
13.02354


In [62]:
pds.eval_series(
    pl.Series(values=np.random.random(size = 1000)), 
    expr = "query_c3_stats", # name of the pds expression
    lag = 3
)

c3_stats
f64
0.137019
